In [ ]:
from Dicom2Nii_Funs import convert_dicom_to_nifty

In [ ]:
import nibabel as nib
import glob
import numpy as np
import re
import pandas as pd
import os
import math
import pydicom as pdcm

In [ ]:
import yaml
with open('config_Dicom2Nii.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
    
    itvtot = config['itv_tot_labels']
    itvtum = config['itv_tumor_labels']
    itvnod = config['itv_ln_labels']
    
    gtvtot = config['gtv_tot_labels']
    gtvtum = config['gtv_tumor_labels']
    gtvnod = config['gtv_ln_labels']

    root_nii = config['save_path']
    
    #source_csv_path = config['source_csv']
    source_csv_path = "//zkh/appdata/RTDicom/Projectline_modelling_lung_cancer/Users/Luis/ListsOfPatients/Dicom2Nii_LastRound.csv"
    
    
    save_path = config['save_path']

df = pd.read_csv(source_csv_path)
target_labels = itvtot + itvtum+itvnod+gtvtot+gtvtum+gtvnod

In [ ]:
df['pxID'] = df['pxID'].astype(int)
id_column = df.iloc[:, 0]
df

In [ ]:
def LookForPlanCT_GTV_ITV_InNii(path_Nii):
    missingCT_bool,missingITV_bool,missingGTV_bool = (True,True,True)
    for f in os.listdir(path_Nii):
        if "ave" in f.lower() or "mip" in f.lower() or "thorax" in f.lower():
            missingCT_bool= False
        if "gtv" in f.lower():
            missingGTV_bool = False
        if "itv" in f.lower() or "igtv" in f.lower():
            missingITV_bool = False    
    return missingCT_bool,missingITV_bool,missingGTV_bool

In [ ]:
count=0
pattern = re.compile(r'(?![2]),(?![0-9])')
for patientID in id_column:
    #COLLECT AND FORMAT THE DATA 
    PlanCT_path = df[id_column==patientID]['CT'].iloc[0]
    PlanCT_path_list = pattern.split(PlanCT_path)
    PlanCT_path_clean = [element.strip() for element in PlanCT_path_list]
    PlanCT_path_unique = list(set(PlanCT_path_clean))
    
    listCTs_path = df[df['pxID'] == patientID].iloc[:,8:20].values.tolist()
    listCTs_path_split = [item for sublist in listCTs_path for item in sublist]

    listRTs_path = df[df['pxID'] == patientID].iloc[:, 2:8].values.tolist()
    listRTs_path_split = [item for sublist in listRTs_path for item in sublist]
    listRTs_path_unique = np.unique(listRTs_path_split)

    #CREATE PATH
    if not os.path.exists(save_path+str(patientID)):
        os.makedirs(save_path+str(patientID)+'/')
        print("NEW Px:",patientID,"Count",count)

    #CHECK IF PX has been completed before
    missPlanCTNii,missITVNii,missGTVTNii = LookForPlanCT_GTV_ITV_InNii(save_path+str(patientID))
    if False and not(missPlanCTNii):
        print("Already Complete Px:",patientID,"Count",count)
    else:
        print("Not complete Folder Px:",patientID,"Count",count)
        save_px_path = os.path.join(save_path,str(patientID))
        
        #Convert Plan CT
        print("Plan CT")
        for singPlanCT_path in PlanCT_path_unique:
            print(singPlanCT_path.split)
            #dicom2nifti.convert_directory(singPlanCT_path, save_px_path, compression=True, reorient=False)
            Filename="PlanCT"
            input_filepaths = []

            for currSlide in os.listdir(singPlanCT_path):
                input_filepaths.append(os.path.join(singPlanCT_path,currSlide))

            image, pixel_spacing, image_position_patient,axial_positions = convert_dicom_to_nifty(input_filepaths,patientID,
                                    save_px_path,
                                    target_labels,extension='.nii.gz',filename="CT_"+Filename,
                                    pixel_spacing = None, image_position_patient=None,axial_positions=None,
                                    np_image=None,dtype_image=np.float32,dtype_mask=np.uint8,)
                
            planCT_Nii_path = os.listdir(save_px_path)

        # RTSTruct
        
        print("RTStruct:")
        ct_nifti = nib.load(os.path.join(save_px_path,planCT_Nii_path[0]))
        for singleRT_path in listRTs_path_unique:
            if not(str(singleRT_path)=="nan"):
                print(singleRT_path)
                try:
                    #rtstructUTILS = RTStructBuilder.create_from(dicom_series_path=singPlanCT_path,rt_struct_path=singleRT_path.replace("\\\\", "/"))
                    Filename="RTstruct"
                    convert_dicom_to_nifty(singleRT_path,patientID,
                                            save_px_path,
                                            target_labels,extension='.nii.gz',filename="RT_"+Filename,
                                            pixel_spacing = pixel_spacing, image_position_patient=image_position_patient,axial_positions=axial_positions,
                                            np_image=image,dtype_image=np.float32,dtype_mask=np.uint8,)
                        
                except Exception as e:
                        print("ERROR at RTUtils",e)
                        pass
                    
    count+=1 

In [ ]:
planCT_Nii_path